##Install all Dependencies


In [ ]:
!pip install pandas
!pip install sentence-transformers
!pip install -q  faiss-cpu torchvision pillow requests
!pip install -q git+https://github.com/openai/whisper.git
!pip install gradio
!sudo apt update && sudo apt install ffmpeg -y

##Import Libraries

In [ ]:
import re
import zipfile
import pandas as pd
import faiss
import numpy as np
import torch
import requests
import whisper
import gradio as gr
from sentence_transformers import SentenceTransformer
from io import BytesIO
from torchvision import transforms
from IPython.display import Image as IPythonImage, display, HTML


In [ ]:
from google.colab import files
files.upload()

In [ ]:
with zipfile.ZipFile("styles.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

In [ ]:
# Use the on_bad_lines='skip' argument to skip problematic lines
df = pd.read_csv("styles.csv", on_bad_lines='skip')
df = df.dropna(subset=["productDisplayName", "gender", "masterCategory"])
df = df.reset_index(drop=True)
df.head()

## Combine Key Product Attributes for Embedding

In [ ]:
df["combined"] = (
    df["productDisplayName"].astype(str) + " " +
    df["gender"].astype(str) + " " +
    df["masterCategory"].astype(str) + " " +
    df["subCategory"].astype(str) + " " +
    df["articleType"].astype(str) + " " +
    df["baseColour"].astype(str) + " " +
    df["season"].astype(str) + " " +
    df["usage"].astype(str)
).str.lower()

df["combined"].head()

##Generate Semantic Embeddings and Build FAISS Index


In [ ]:
# Load the transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert product descriptions into vector embeddings
embeddings = model.encode(df["combined"].tolist(), convert_to_numpy=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 = Euclidean distance
index.add(embeddings)

In [ ]:
def search_products(query, top_k=5):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)
    results = df.iloc[I[0]].copy()
    results["similarity_score"] = D[0]
    return results[["id", "productDisplayName", "gender", "masterCategory", "subCategory", "articleType", "baseColour", "season", "year", "usage", "similarity_score"]]


In [ ]:
search_products("white shirt for men")

In [ ]:
search_products("black sneakers for men")

In [ ]:
search_products("summer cotton dress")

In [ ]:
search_products("formal shirt for office wear")

In [ ]:
search_products("watch under 4000")

In [ ]:
from google.colab import files
files.upload()

In [ ]:
with zipfile.ZipFile("images.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

In [ ]:
images_df = pd.read_csv("images.csv")
images_df.head()

##Merge Product Metadata with Image URLs

In [ ]:
# Merge with product DataFrame
df["id"] = df["id"].astype(int)
# Assume 'filename' column in images_df contains the product ID
# Extract the ID from the filename, assuming it's in the format 'id_...' or 'id.jpg'
images_df["id"] = images_df["filename"].str.split("_|.jpg").str[0].astype(int) # Split by '_' or '.jpg' to handle different filename formats

df = pd.merge(df, images_df, on="id", how="left")

# Store image link
df["image_url"] = df["link"]
df[["id", "productDisplayName", "image_url"]].head()

##Load CLIP Model for Image Embeddings

In [ ]:
# Load CLIP model
clip_model = SentenceTransformer('clip-ViT-B-32')

# Define image transform
clip_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                         std=[0.26862954, 0.26130258, 0.27577711])
])

##Generate Image Embeddings for Visual Search

In [ ]:
from PIL import Image

image_vectors = []
valid_indices = []

for i, row in df.head(1000).iterrows():
    try:
        response = requests.get(row["image_url"], timeout=3)
        img = Image.open(BytesIO(response.content)).convert("RGB")

        # Encode the image using SentenceTransformer's encode
        embedding = clip_model.encode(img, convert_to_tensor=False)
        image_vectors.append(embedding)
        valid_indices.append(i)
    except Exception as e:
        print(f"Skipping image at index {i} due to error: {e}")

In [ ]:
image_vectors_np = np.vstack(image_vectors).astype("float32")

visual_index = faiss.IndexFlatL2(image_vectors_np.shape[1])
visual_index.add(image_vectors_np)

##Define Visual Search Function (Image → Product)

In [ ]:
def visual_search(user_image, top_k=5):
    img = Image.open(user_image).convert("RGB")

    # Encode using SentenceTransformer
    user_vec = clip_model.encode(img, convert_to_tensor=False).reshape(1, -1).astype("float32")

    D, I = visual_index.search(user_vec, top_k)

    matched_rows = [valid_indices[idx] for idx in I[0]]
    results = df.iloc[matched_rows].copy()
    results["similarity_score"] = D[0]

    return results[["productDisplayName", "gender", "masterCategory", "subCategory", "articleType", "image_url", "similarity_score"]]


## upload image

In [ ]:
from google.colab import files
uploaded = files.upload()

for fname in uploaded:
    results = visual_search(fname)
    display(results)

##Display Visual Search Results as Product Cards

In [ ]:
def show_visual_results(results):
    html = ""
    for _, row in results.iterrows():
        html += f"""
        <div style="display:flex; align-items:center; margin-bottom:10px;">
            <img src="{row['image_url']}" width="100" style="margin-right:10px;">
            <div>
                <b>{row['productDisplayName']}</b><br>
                {row['masterCategory']} / {row['subCategory']}<br>
                <i>{row['gender']}</i><br>
                <b>Similarity:</b> {row['similarity_score']:.2f}
            </div>
        </div>
        """
    display(HTML(html))


In [ ]:
for fname in uploaded:
    results = visual_search(fname)
    show_visual_results(results)


## Upload Audio

In [ ]:
from google.colab import files
uploaded_audio = files.upload()

##Transcribe Voice Input Using Whisper model by OpenAI

In [ ]:
whisper_model = whisper.load_model("base")

# Get the uploaded filename
audio_file = list(uploaded_audio.keys())[0]

# Transcribe to text
transcription = whisper_model.transcribe(audio_file,language="en")
query_text = transcription["text"]

print(" Voice input:", query_text)

# Reload or re-assign your SentenceTransformer model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
results = search_products(query_text)
display(results)

##Extract Filters from Natural Language Queries


In [ ]:
def extract_filters(query):
    filters = {"gender": None, "category": None, "usage": None, "max_price": None}
    query = query.lower()

    # Gender detection
    if "women" in query or "ladies" in query:
        filters["gender"] = "Women"
    elif "men" in query or "gents" in query:
        filters["gender"] = "Men"

    # Usage detection
    if "casual" in query:
        filters["usage"] = "Casual"
    elif "formal" in query:
        filters["usage"] = "Formal"
    elif "sports" in query:
        filters["usage"] = "Sports"

    # Price detection
    price_match = re.search(r"under (\d+)", query)
    if price_match:
        filters["max_price"] = int(price_match.group(1))

    return filters

##Perform Semantic Search with Smart Filters

In [ ]:
def search_products_with_filters(query, gender=None, category=None, usage=None, max_price=None, top_k=10):
    # Step 1: Semantic search
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)
    results = df.iloc[I[0]].copy()
    results["similarity_score"] = D[0]

    # Step 2: Apply smart filters
    if gender:
        results = results[results["gender"].str.lower() == gender.lower()]
    if category:
        results = results[results["masterCategory"].str.lower() == category.lower()]
    if usage:
        results = results[results["usage"].str.lower() == usage.lower()]
    if max_price:
        results = results[results["price"] <= max_price]

    return results.sort_values(by="similarity_score")[[
        "productDisplayName", "gender", "masterCategory", "subCategory", "usage", "price", "similarity_score","image_url"
    ]]


##Combine Everything into a Smart Search Function

In [ ]:
def smart_search(query, top_k=10):
    filters = extract_filters(query)

    print(" Auto-detected filters:")
    print(f"  Gender: {filters['gender']}")
    print(f"  Usage: {filters['usage']}")
    print(f"  Max Price: {filters['max_price']}")

    return search_products_with_filters(
        query=query,
        gender=filters["gender"],
        category=filters["category"],  # You can expand this later
        usage=filters["usage"],
        max_price=filters["max_price"],
        top_k=top_k
    )


##Simulate Product Prices for Filter-Based Search

In [ ]:
# Add random price between ₹500 and ₹4999
np.random.seed(42)  # Ensures reproducible results
df["price"] = np.random.randint(500, 5000, size=len(df))

# Check a few rows
df[["productDisplayName", "price"]].head()


In [ ]:
smart_search("white kurta for women under 1500")

In [ ]:
smart_search("denim jeans for men under 1500")

In [ ]:
smart_search("formal shoes for men under 2000")

## Product Curation

In [ ]:
def ai_curation(query, top_k=30):
    # Step 1: Run smart search (with auto filters)
    base_results = smart_search(query, top_k=top_k)

    # Step 2: Curation logic
    top_picks = base_results.head(5)
    budget_friendly = base_results[base_results["price"] < 1000].head(5)

    # Optional: Use a color keyword from the query (e.g., "white")
    import re
    color_match = re.search(r"\b(white|black|blue|red|green|yellow|pink)\b", query.lower())
    color = color_match.group(1) if color_match else None

    if color:
        similar_styles = base_results[base_results["productDisplayName"].str.contains(color, case=False)].head(5)
    else:
        similar_styles = base_results.head(5)

    # Optional personalization (dummy: show casual for women)
    personalized = base_results[
        (base_results["gender"] == "Women") & (base_results["usage"] == "Casual")
    ].head(5)

    return {
        " Top Picks": top_picks,
        " Budget Friendly": budget_friendly,
        f" Similar Styles ({color or 'default'})": similar_styles,
        " For You (Personalized)": personalized
    }


In [ ]:
def show_curated_carousels(carousels):
    for title, df_section in carousels.items():
        html = f"<h3>{title}</h3><div style='display:flex; overflow-x:auto;'>"
        for _, row in df_section.iterrows():
            html += f"""
            <div style="margin-right:10px; text-align:center;">
                <img src="{row['image_url']}" width="120" style="border-radius:8px;"><br>
                <b>{row['productDisplayName'][:25]}</b><br>
                ₹{row['price']} | {row['usage']}<br>
            </div>
            """
        html += "</div><hr>"
        display(HTML(html))


In [ ]:
carousels = ai_curation("Running shoes for women under 4500")
show_curated_carousels(carousels)


##Gradio Frontend

In [ ]:
import gradio as gr

# Load Whisper model for audio transcription
whisper_model = whisper.load_model("base")


def gradio_search(text_query, image=None, audio=None):
    # Step 1: Transcribe audio if no text query is given
    if not text_query.strip() and audio is not None:
        transcription = whisper_model.transcribe(audio)
        text_query = transcription["text"]
        print(" Transcribed:", text_query)

    # Step 2: Choose search type
    if image is not None and not text_query.strip():
        results = visual_search(image)
    else:
        results = smart_search(text_query)

    # Step 3: Prepare Gradio output
    results = results.head(6)
    items = [
        f"{row['productDisplayName']}\n₹{row['price']} | {row['gender']} | {row['usage']}\nScore: {row['similarity_score']:.2f}"
        for _, row in results.iterrows()
    ]
    return list(results["image_url"]), items

# --- Gradio UI Layout ---
with gr.Blocks() as demo:
    gr.Markdown("""
    # ONDC Smart Discovery Engine
    Search using voice, text, or image. AI will match the most relevant products across sellers.
    """)

    with gr.Row():
        text_input = gr.Textbox(label="Search Query", placeholder="e.g. white kurta for women under 1500")
        image_input = gr.Image(type="pil", label="Upload Product Image (Optional)")
        audio_input = gr.Audio(type="filepath", label="Upload Voice Query (Optional)")

    search_btn = gr.Button("Search")

    with gr.Row():
        result_images = gr.Gallery(label="Matching Products")
        result_texts = gr.Textbox(label="Product Info", lines=10)

    search_btn.click(fn=gradio_search, inputs=[text_input, image_input, audio_input], outputs=[result_images, result_texts])

# Launch the app
demo.launch(debug=False, share=True)